In [1]:
from os import listdir
from os.path import isfile, join

mypath = "/Volumes/TOSHIBA/Rekordbox_database"

files = [f for f in listdir(mypath) if not (f).startswith(".")]

In [2]:
import requests
import discogs_client
d = discogs_client.Client('tracklist_explorer', user_token='TQnLrfoPbUSIPjrwWqIkZSkuJczEwsHtQXbdegwL')

In [3]:
import pandas as pd

df = pd.DataFrame(files, columns=["title"])

In [4]:
df

,title
0,SPIRINET - ENERGY FORMULA -KHT007
1,Babe Roots feat Kojo Neatness - Be Still - Raw...
2,CASH FROM HASH - CASH FROM HASH - AXON
3,Jay Glass Dubs - Epitaph
4,"Mina, Bryte, King Doudou - Mina ft. Bryte - Se..."
...,...
648,Squito Babe - Palombaro [Explicit]
649,RSD-Corner_Dub_(Blue_and_Red_Mix)_&_Pretty_Bri...
650,Minor Science - Whities 008 [Ep] (2016)
651,(1997) Dj Jubilee - Eternal Dark (Unknown Grou...


In [5]:
import re

In [6]:
df["title"] = [x[:-4] if x.endswith(".mp3") else x for x in df["title"]]
df["title"] = [x.replace("feat", "") for x in df["title"]]
df["title"] = [x.replace("'", "") for x in df["title"]]
df["title"] = [x.replace("\"", "") for x in df["title"]]
df["title"] = [x.replace("-", " ") for x in df["title"]]
df["title"] = [x.replace("_", " ") for x in df["title"]]
df["title"] = [x.replace("&", " ") for x in df["title"]]
df["title"] = [re.sub("[\(\[].*?[\)\]]", "", x) for x in df["title"]]

In [7]:
df

,title
0,SPIRINET ENERGY FORMULA KHT007
1,Babe Roots Kojo Neatness Be Still Rawness
2,CASH FROM HASH CASH FROM HASH AXON
3,Jay Glass Dubs Epitaph
4,"Mina, Bryte, King Doudou Mina ft. Bryte Se..."
...,...
648,Squito Babe Palombaro
649,RSD Corner Dub Pretty Bright Light
650,Minor Science Whities 008
651,Dj Jubilee Eternal Dark


In [8]:
df.to_csv("df_clean_titles.csv")

works with single tracks as well

In [9]:
release = d.search('Clara! y Maoupa - Truenos', type='release')[0]

In [10]:
release.styles

['Experimental', 'Reggaeton']

In [11]:
styles = []
genres = [] 
for idx,i in df.iterrows():
    try:
        release = d.search(i["title"], type='release')[0]
        styles.append(release.styles)
        genres.append(release.genres)
    except:
        styles.append("error")
        genres.append("error")

In [12]:
df["style"] = styles
df["genres"] = genres

In [13]:
df.to_csv("df.csv")

In [14]:
df["style"].value_counts()

error                                             119
[Techno]                                           34
[Dubstep]                                          34
[House]                                            11
[Experimental]                                      7
                                                 ... 
[Electro, Techno, Industrial, Acid, Breakbeat]      1
[Bass Music, House]                                 1
[Grime, Dub Techno, Breaks, Leftfield]              1
[Downtempo, EBM, Dub]                               1
[Experimental, Glitch]                              1
Name: style, Length: 352, dtype: int64

In [15]:
errors = df[df["style"] == "error"]

In [16]:
len(errors)

119

In [17]:
errors.to_csv("errors.csv")

In [81]:
df.loc[errors.index].drop("clean_title", axis=1)

,title,style
1,Babe Roots Kojo Neatness - Be Still - Rawness,Error
2,CASH FROM HASH - CASH FROM HASH - AXON,Error
4,"Mina, Bryte, King Doudou - Mina ft. Bryte - Se...",Error
7,scopeotaku_4040_d,Error
11,King Doudou - Toa,Error
...,...,...
651,Loop_LF--Natural_XT_EP-WSRLF2-WEB-2019-BABAS,Error
652,ArtSaves - Home Is Where the Hatred Is,Error
654,Abu AMA - INTERNATiONAL ROKKAZ,Error
658,Little Artist - CANCEL COPENHAGEN - 01 Cancel ...,Error


In [82]:
styles = []
genres = [] 
for idx,i in df.loc[errors.index].iterrows():
    try:
        release = d.search(i["title"], type='release')[0]
        styles.append(release.styles)
        genres.append(release.genres)
    except:
        styles.append("release not found")
        genres.append("release not found")